## Day 47 Lecture 2 Assignment

In this assignment, we will perform K-Medoids clustering using responses to a survey about student life at a university.

In [2]:
! pip install pyclustering

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from pyclustering.cluster.kmedoids import kmedoids
import random
%matplotlib inline

This dataset consists of 35 binary features, each corresponding to a yes/no question that characterizes the student taking the survey.

This dataset contains a large number of features, each corresponding to a survey question. The feature name summarizes the survey question, so we will not list them all out here.

Load the dataset.

In [3]:
survey = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Data%20Sets%20Clustering/student_life_survey.csv')
survey.head()

,Q2-Participated in Societies and Interest Groups,Q2-Participated in Clubs,"Q2-Participated in Halls, JCRCs and/or Residential College CSCs",Q2-Participated in University organised events,Q3-Interested in Arts & Culture,Q3-Interested in Science & Technology,Q3-Interested in Research and independent study,Q3-Interested in Sports,"Q3-Interested in Other competitions (eg case, debates)",Q3-Interested in Entrepreneurship,...,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others,response_id
0,0,1,0,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,2
2,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,3
3,1,1,1,1,0,1,1,0,0,0,...,1,0,1,1,1,1,0,1,0,4
4,1,0,1,1,0,1,1,0,0,1,...,1,1,0,1,0,0,0,1,0,5


For our analysis, we will focus on a specific subset of the survey that is focused on stress. These questions all begin with the string 'Q5'. Filter the columns that meet this criteria (should be 10 in total).

In [4]:
survey.index = survey.response_id
survey = survey.filter(regex='Q5')
survey.head()

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
response_id,,,,,,,,,,
1,0,1,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0
4,1,1,0,1,1,1,1,0,1,0
5,0,1,1,0,1,0,0,0,1,0


The pyclustering implementation of kmedoids supports a variety of distance metrics, but they are primarily for numeric data. We will be using SMC/Hamming dissimilarity and precomputing the similarity matrix (an alternative would be to specify a user-defined function, which you are welcome to try in addition).

We'll assume for the next step that a pair of negative values (i.e. both responses are "no") is as informative as a pair of positive values. Compute the full distance/dissimilarity matrix for the survey data using matching/hamming distance.

In [5]:
sim = pdist(survey, metric='matching')
sim = pd.DataFrame(squareform(sim), index=survey.index, columns=survey.index)
sim.head()

response_id,1,2,3,4,5,6,7,8,9,10,...,2949,2950,2951,2952,2953,2954,2955,2956,2957,2958
response_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
2,0.0,0.0,0.1,0.6,0.3,0.1,0.2,0.3,0.4,0.4,...,0.1,0.3,0.2,0.3,0.3,0.2,0.0,0.0,0.2,0.2
3,0.1,0.1,0.0,0.7,0.4,0.2,0.1,0.2,0.5,0.3,...,0.2,0.4,0.3,0.4,0.4,0.3,0.1,0.1,0.1,0.1
4,0.6,0.6,0.7,0.0,0.5,0.5,0.8,0.7,0.2,0.6,...,0.7,0.5,0.4,0.5,0.3,0.6,0.6,0.6,0.6,0.8
5,0.3,0.3,0.4,0.5,0.0,0.2,0.3,0.6,0.3,0.3,...,0.2,0.2,0.3,0.4,0.2,0.3,0.3,0.3,0.5,0.3


Using the dissimilarity matrix, perform kmedoids clustering using k=2. Set the initial medoids randomly. Note that pyclustering expects the distance matrix to be a numpy array; a pandas dataframe may cause errors. 

Which survey responses are chosen as the cluster representatives? Print out the details of these responses.

In [6]:
n_clusters = 2
initial_index_medoids = random.sample(list(sim.index), n_clusters)

sim_kmedoids = np.delete(sim.values, sim.values.shape[1] - 1, 1)  # removes the index column from the array

kmedoids_instance = kmedoids(sim_kmedoids, initial_index_medoids = initial_index_medoids, data_type='distance_matrix')
kmedoids_instance.process()
medoids = list(map(lambda x: x+1, kmedoids_instance.get_medoids())) # add 1 to recover original response_ids
    
survey.loc[medoids]

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
response_id,,,,,,,,,,
2958,0,1,1,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0,1,0,0


If you run the previous cell a few times, you'll probably notice that the medoids are very sensitive to initialization. A common approach to produce well-separated clusters is to choose initial centroids that are far apart. Re-run the previous process, except with a random pair of centroids that have a dissimilarity of 0.8 or higher. Are the results more stable now? How would you describe the typical clusters you see?

In [7]:
sim['id'] = sim.index
sim_melted = pd.melt(sim, id_vars=['id'], value_vars=sim.columns[sim.columns != 'id'])
sim_melted.columns = ['response_id_1', 'response_id_2', 'dissimilarity']

cutoff = 0.8
sim_melted_subset = sim_melted[(sim_melted.response_id_1 < sim_melted.response_id_2) 
                               & (sim_melted.dissimilarity >= cutoff)]  # removes all duplicate and self-pairs, and dissim lower than 0.8

initialize = random.sample(list(sim_melted_subset.index), 1)
initial_index_medoids = [sim_melted_subset.loc[initialize].response_id_1.min() - 1,
                         sim_melted_subset.loc[initialize].response_id_2.min() - 1] # subtract 1 to convert response_ids into array indices

sim_kmedoids = np.delete(sim.values, sim.values.shape[1] - 1, 1)  # removes the index column from the array

kmedoids_instance = kmedoids(sim_kmedoids, initial_index_medoids = initial_index_medoids, data_type='distance_matrix')
kmedoids_instance.process()
medoids = list(map(lambda x: x+1, kmedoids_instance.get_medoids())) # add 1 to recover original response_ids
    
survey.loc[medoids]

,Q5-Stressed about Adjustment issues,Q5-Stressed about Academic issues,Q5-Stressed about Financial issues,Q5-Stressed about Family issues,Q5-Stressed about Friendships,Q5-Stressed about Romantic relationships,Q5-Stressed about Health related issues,Q5-Stressed about Career related issues,"Q5-Stressed about My involvement in hostel, clubs, societies, interest groups, etc.",Q5-Stressed about Others
response_id,,,,,,,,,,
1,0,1,0,0,0,0,0,0,0,0
257,1,1,1,0,1,0,0,1,0,0


In [8]:
sim_melted

,response_id_1,response_id_2,dissimilarity
0,1,1,0.0
1,2,1,0.0
2,3,1,0.1
3,4,1,0.6
4,5,1,0.3
...,...,...,...
8749759,2954,2958,0.2
8749760,2955,2958,0.2
8749761,2956,2958,0.2
8749762,2957,2958,0.2


Although there is still some variability from run to run, the overall pattern is much clearer: one cluster of students who are stressed out about several issues and a second cluster of students who are only stressed about academic issues.